In [65]:
import os
import time
import copy
import pickle
import itertools
import numpy as np
import pandas as pd

import multiprocessing as mp
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Show that when using just Numberbatch you get worse results

In [3]:
# from utils import get_word_neighborhood, filter_neighborhood

In [53]:
args_cach_path = '/data/zeste_cache/neighborhoods/'
args_prefetch_path = '/data/zeste_cache/prefetch_neighborhoods/'
args_numberbatch_pickle = '/home/semantic/harrando/zeste/numberbatch-en-19.08-en.pickle'
args_dataset_csv_path = 'datasets/bbc_dataset.csv'
args_labels_mapping = '20ng_labels_mapping.txt'

In [5]:
numberbatch = pickle.load(open(args_numberbatch_pickle, 'rb'))

In [6]:
nb_sim = numberbatch.similarity

In [7]:
lemmatizer = WordNetLemmatizer()

In [54]:
df_dataset = pd.read_csv(args_dataset_csv_path)
raw_corpus = df_dataset.text.tolist()
gt_labels = df_dataset.label.tolist()
unique_labels = sorted(set(gt_labels))

unique_labels

['business', 'entertainment', 'politics', 'sport', 'tech']

In [121]:
ngram_counter = TfidfVectorizer(ngram_range=(1,3), min_df=2)
ngram_counter.fit([' '.join(preprocess(d)) for d in raw_corpus])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=2, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [123]:
confusion_matrix

In [124]:
ngrams[:10]

['vice versa',
 'business world',
 'past present',
 'video game',
 'also used',
 'digital technology',
 'take photo',
 'last year',
 'picture message',
 'big business']

In [55]:
labels_mapping = dict(l.strip().split('\t') for l in open(args_labels_mapping))
labels = sorted(set(labels_mapping.values()))

In [56]:
labels

['baseball',
 'car',
 'cryptography',
 'electronics',
 'graphic',
 'gun',
 'hardware',
 'hockey',
 'medicine',
 'middle_east',
 'motorcycle',
 'politics',
 'religion',
 'sale',
 'space',
 'windows']

In [71]:
def get_word_neighborhood(label, depth=1, numberbatch=numberbatch, cache_path=args_cach_path, prefetch_path=args_prefetch_path):
    # In case the requested label does not appear in the cache
    
    pickle_path = os.path.join(args_cach_path, label+'.pickle')
    if depth == 0 or not os.path.exists(pickle_path) or label not in numberbatch:
        return {}
    
    # if already computed
    prefetch_path = os.path.join(prefetch_path, str(depth) + '/' + label+'.pickle')
    if depth > 1 and os.path.exists(prefetch_path):
        # print('Prefetching the pickle for label', label, '..')
        return pickle.load(open(prefetch_path, 'rb'))

    # Get immediate label neighborhood
    similarities = ['simple', 'compound', 'depth', 'harmonized']
    neighborhood = pickle.load(open(pickle_path, 'rb'))
    for node in neighborhood:
        # we add the possiblity of defining multiple similarity methods for nodes that are not directly connected
        # to the Label node
        neighborhood[node]['rels'] = [tuple(neighborhood[node]['rels'])]
        neighborhood[node]['sim'] = {sim:neighborhood[node]['sim'] for sim in similarities}
    
    # Connect to n-hops labels
    hops = 1
    to_visit_next = list(neighborhood.keys())
    while hops < depth:
        next_hop = []
        while len(to_visit_next) > 0:
            current_node = to_visit_next.pop()
            if current_node in stopwords.words('english'):
                continue
            cnn = get_word_neighborhood(current_node, depth=1)
            for word in cnn:
                if word not in neighborhood:
                    neighborhood[word] = {'from':[], 'rels': [], 'sim':{}}
                    sim_dict = {sim: 0.0 for sim in similarities}
                else:
                    sim_dict = neighborhood[word]['sim']
                    
                neighborhood[word]['from'].append(current_node)
                neighborhood[word]['rels'].append(tuple(cnn[word]['rels']))
                if word in numberbatch:
                    sim_label_word = numberbatch.similarity(label, word)
                    sim_dict['simple'] = max(sim_dict['simple'], sim_label_word)
                    sim_dict['depth']  = max(sim_dict['depth'], sim_label_word / (hops + 1))
                    if current_node in numberbatch:
                        sim_cn_word = numberbatch.similarity(current_node, word)
                        sim_label_cn = numberbatch.similarity(label, current_node)
                        compound = sim_label_cn * sim_cn_word
                        harmonized = 2*compound / (sim_label_cn + sim_cn_word)
                        sim_dict['compound'] = max(sim_dict['compound'], compound)
                        sim_dict['harmonized'] = max(sim_dict['harmonized'], harmonized)
                    else:
                        sim_dict['compound'] = max(sim_dict['compound'], sim_label_word)
                        sim_dict['harmonized'] = max(sim_dict['harmonized'], sim_label_word)
            
                # print('From label' + label + '- current node:' + current_node + ', word:' + word)
                neighborhood[word]['sim'] = sim_dict
                next_hop.append(word)
        
        hops += 1
        to_visit_next = next_hop
        
    # save 
    if depth > 1:
        pickle.dump(neighborhood, open(prefetch_path, 'wb'))

    return neighborhood

In [74]:
def get_label_neighborhood(label_words, depth=2):

    ns = []
    words = label_words.split(';')
    for word in words:
        ns.append(get_word_neighborhood(word, depth=depth))
    neighborhood = ns[0].copy()
    
    for current_node, cnn in zip(words[1:], ns[1:]):
        for word in cnn:
            if word in neighborhood:
                neighborhood[word]['from'].append(current_node)
                neighborhood[word]['rels'].append(tuple(cnn[word]['rels']))
                neighborhood[word]['sim'] = {s:max(neighborhood[word]['sim'][s], cnn[word]['sim'][s]) for s in cnn[word]['sim']}
            else:
                neighborhood[word] = {}
                neighborhood[word]['from'] = [current_node]
                neighborhood[word]['rels'] = [tuple(cnn[word]['rels'])]
                neighborhood[word]['sim']  = cnn[word]['sim'].copy()

    return neighborhood

In [118]:
def preprocess(document):
    document = document.replace("'ll", ' will').replace("s' ", 's').replace("'s", '').replace("-", '_')
    document = ''.join(c for c in document if c not in '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~')
    document = [w for w in document.lower().split(' ') if w not in stopwords.words('english')]
    document = [lemmatizer.lemmatize(w) for w in document if w != '']
    return document

In [98]:
def filter_neighborhoood(neighborhood_original, allowed_rels, sim, keep):
    if allowed_rels == 'all' and sim == 'simple' and keep == 'all':
        return neighborhood_original

    neighborhood = copy.deepcopy(neighborhood_original)
    
    if allowed_rels == 'related':
        allowed_rels = ['DefinedAs', 'DerivedFrom', 'HasA', 'InstanceOf', 'IsA', 'PartOf', 'RelatedTo', 'SimilarTo', 'Synonym', 'Antonym']
    
    if allowed_rels != 'all':
        nodes = list(neighborhood.keys())
        for node in nodes:
            if not any(rel in rels for rel in allowed_rels for rels in neighborhood[node]['rels']):
                del neighborhood[node]
                continue
    
    if keep != 'all':
        all_scores = sorted([neighborhood[node]['sim'][sim] for node in neighborhood], reverse=True)
        if keep.startswith('top') and keep.endswith('%'):
            keep = int(keep[3:-1]) / 100.
            cutoff_score = all_scores[int(keep*(len(all_scores)))-1]
        elif keep.startswith('top'):
            keep = int(keep[3:])
            cutoff_score = all_scores[min(len(all_scores)-1,keep)]
        elif keep.startswith('thresh'):
            cutoff_score = float(keep[6:])
        nodes = list(neighborhood.keys())
        for node in nodes:
            node_sim = neighborhood[node]['sim'][sim]
            if node_sim <= cutoff_score:
                del neighborhood[node]
                continue

    return neighborhood

In [144]:
space_2 = labels_at_depth[2]['space']

In [139]:
gun_2 = labels_at_depth[2]['gun']

In [145]:
lns2 = {'space': space_2, 'gun': gun_2}

In [110]:
tech_2['hierological']['sim']

{'simple': 0.75864804,
 'compound': 0.7698828,
 'depth': 0.3793240189552307,
 'harmonized': 0.870072858764631}

In [102]:
r = filter_neighborhoood(tech_2, 'all', 'simple', 'top100')
len(r)

100

In [18]:
labels_at_depth = {}

for depth in [2, 3]:
    print('For depth ', depth, ':')
    labels_at_depth[depth] = {}
    for label in tqdm(labels): 
        labels_at_depth[depth][label] = get_label_neighborhood(label, depth)import pickle

For depth  2 :



For depth  3 :


In [187]:
def score(tokens, label_neighborhood, sim, ngrams, normalize):   
    if ngrams:
        doc = ' '.join(tokens)
        for ngram in ngrams:
            if ngram in doc:
                tokens.append(ngram)
    
    score = 0
    inter = 0
    for token in tokens:
        if token in label_neighborhood:
            score += label_neighborhood[token]['sim'][sim]
            inter += 1 
            
    if normalize == 'inter_len':
        return round(score / max(inter, 1), 6)
    elif normalize == 'max_score':
        return round(score / sum([label_neighborhood[node]['sim'][sim] for node in label_neighborhood]), 6)
    else:
        return round(score, 6)

In [188]:
score(preprocess('I love some space video game technology!'), tech_2, 'simple', ngrams=ngrams, normalize='')

1.253939

In [173]:
data = [preprocess(d) for d in ['I love science', 'I love weapons', 'I love time travel', 'I love war']]

In [192]:
def predict_dataset(docs, sorted_labels, labels_neighborhoods, sim, ngrams, normalize):
    scores = np.zeros((len(docs), len(sorted_labels)))
    for i, doc in enumerate(docs):
        for j, label in enumerate(sorted_labels):
            scores[i][j] = score(doc, labels_neighborhoods[label], sim, ngrams, normalize)
    return scores

In [193]:
sorted_labels = [ 'gun', 'space', 'war;gun' ]
p_matrix = predict_dataset(data, sorted_labels, lns2, 'harmonized', ngrams=ngrams, normalize='inter_len')
p_matrix

array([[0.120803, 0.400857, 0.212052],
       [0.474738, 0.220495, 0.548871],
       [0.116798, 0.432931, 0.310222],
       [0.357784, 0.188647, 0.614708]])

In [191]:
preds = np.argmax(p_matrix, axis=1)
predicted_labels = [sorted_labels[p] for p in preds]
predicted_labels

['space', 'war;gun', 'space', 'war;gun']

In [186]:
lns2['war;gun'] = get_label_neighborhood('war;gun', 2)

Prefetching the pickle for label war ..
Prefetching the pickle for label gun ..


In [201]:
print(evaluate(predicted_labels, [['space', 'gun'], ['war;gun', 'gun'], ['space'], ['war;gun', 'gun']]))

              precision    recall  f1-score   support

       space       1.00      1.00      1.00         2
     war;gun       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

